In [1]:
import isri_tools

<module 'align' from '/work/carlinet/dev/directory-evaluation-tools/thirdparty/ocr-evaluation-tools/align.cpython-39-x86_64-linux-gnu.so'>

In [18]:
a = "👉 étagères𐌰"
b = "etag.µbleble"

A, B = isri_tools.align(a, b, 'x')
print(A)
print(B)

11


In [19]:
A, B = isri_tools.get_alignment_map(a, b)
print(A)
print(B)

👉 étagèrxxes𐌰x
exxtag.µbleble
